## 3. Sample Dataset

Instead of reading from external files, we'll create sample data inline. This dataset simulates examples with varying difficulty levels that influence error probability.

In [ ]:
@dataclass
class DKWController:
    """DKW-guided fusion/fission controller."""
    epsilon_target: float = 0.10  # Target error rate (10%)
    delta: float = 0.05           # Confidence parameter (5%)
    min_samples: int = 100        # Minimum samples before decisions
    hysteresis: float = 0.05      # Hysteresis to prevent oscillation

    samples: list = field(default_factory=list)
    current_state: str = "fission"  # Start conservatively

    def dkw_epsilon(self, n: int) -> float:
        """Compute DKW epsilon for n samples."""
        if n < 2:
            return 1.0
        return np.sqrt(np.log(2 / self.delta) / (2 * n))

    def add_observation(self, error: float) -> None:
        """Add error observation for calibration."""
        self.samples.append(error)

    def decide(self) -> str:
        """Make fusion/fission decision with DKW guarantee."""
        n = len(self.samples)
        if n < self.min_samples:
            return self.current_state

        epsilon = self.dkw_epsilon(n)
        empirical_error = np.mean(self.samples[-self.min_samples:])
        error_upper_bound = empirical_error + epsilon

        if self.current_state == "fusion":
            if error_upper_bound > self.epsilon_target + self.hysteresis:
                self.current_state = "fission"
        else:  # current_state == "fission"
            if error_upper_bound < self.epsilon_target - self.hysteresis:
                self.current_state = "fusion"

        return self.current_state

# Test the controller instantiation
controller = DKWController()
print(f"✅ Controller created with target error rate: {controller.epsilon_target:.1%}")
print(f"   Initial state: {controller.current_state}")
print(f"   Confidence level: {1-controller.delta:.1%}")

## 2. DKW Controller Class

The `DKWController` implements an adaptive decision-making system that switches between "fusion" and "fission" modes based on observed error rates. It uses the **Dvoretzky-Kiefer-Wolfowitz inequality** to provide statistical confidence bounds.

**Key Parameters:**
- `epsilon_target`: Target error rate threshold (10%)
- `delta`: Confidence level parameter (5%)
- `min_samples`: Minimum samples before making decisions
- `hysteresis`: Prevents oscillation between states

In [ ]:
"""DKW Controller Implementation."""
import json
import numpy as np
from dataclasses import dataclass, field
import pandas as pd
import matplotlib.pyplot as plt

# Set random seed for reproducible results
np.random.seed(42)

print("✅ Dependencies imported successfully!")

## 1. Import Dependencies

# DKW Controller Implementation Demo
## Interactive Notebook for Fusion/Fission Decision Making

This notebook demonstrates a **DKW-guided fusion/fission controller** that makes adaptive decisions based on error observations with statistical guarantees. The controller uses the Dvoretzky-Kiefer-Wolfowitz (DKW) inequality to provide confidence bounds on empirical error rates.

**Key Features:**
- Adaptive fusion/fission decision making
- Statistical confidence bounds using DKW inequality
- Hysteresis to prevent decision oscillation
- Self-contained demonstration with sample data